In [6]:
# importing libraries
import pandas as pd
import numpy as np
from numpy import ndarray
from pandas import DataFrame
from sklearn.metrics.pairwise import pairwise_distances


In [7]:
rating_columns = ["user_id", "movie_id", "rating", "unix_timestamp"]
ratings: DataFrame = pd.read_csv("data/ml-100k/u.data", sep="\t", names=rating_columns, encoding="latin-1")

In [8]:
# Ratings Data
print("\nRatings Data :")
print("shape : ", ratings.shape)
ratings.head()


Ratings Data :
shape :  (100000, 4)


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [9]:
number_of_users = ratings.user_id.unique().shape[0]
number_of_movies = ratings.movie_id.unique().shape[0]

In [29]:
data_matrix: ndarray = np.zeros((number_of_users, number_of_movies))
for line in ratings.itertuples():
    data_matrix[line[1] - 1, line[2] - 1] = line[3]
data_matrix_df = DataFrame(data_matrix)
data_matrix_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
def predict(ratings: ndarray, similarity: ndarray, type: str = 'user') -> ndarray:
    pred = None
    # we are taking mean of axis=1 / rows
    # where axis0 -> y-axis and axis1 -> x-axis
    # i.e we are taking means of each row
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array(
            [np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [25]:
user_similarity = pairwise_distances(data_matrix, metric='cosine')
movie_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [27]:
user_prediction = predict(data_matrix, user_similarity, type='user')
DataFrame(user_prediction).head()

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,2.065326,0.734303,0.629924,1.010669,0.640686,0.476150,1.784569,1.163032,1.513350,0.704478,...,0.394041,0.394434,0.393981,0.392972,0.393344,0.392272,0.394909,0.393590,0.393049,0.392771
1,1.763088,0.384040,0.196179,0.731538,0.225643,0.003892,1.493597,0.876153,1.108467,0.261991,...,-0.086942,-0.085491,-0.087137,-0.088158,-0.087298,-0.089288,-0.087468,-0.088378,-0.086918,-0.086712
2,1.795904,0.329047,0.158829,0.684154,0.173277,-0.035621,1.488230,0.835769,1.135426,0.236383,...,-0.134795,-0.133537,-0.135543,-0.136438,-0.135041,-0.137611,-0.136374,-0.136992,-0.134969,-0.134765
3,1.729951,0.293913,0.127741,0.644932,0.142143,-0.062261,1.437010,0.796249,1.096663,0.211789,...,-0.161413,-0.160220,-0.161542,-0.162586,-0.161634,-0.163877,-0.162283,-0.163080,-0.161442,-0.161248
4,1.796651,0.454474,0.354422,0.763130,0.359539,0.195987,1.547370,0.908904,1.292027,0.437954,...,0.101762,0.102405,0.101923,0.100839,0.101711,0.099951,0.102515,0.101233,0.101075,0.101201
